In [1]:
import xml.etree.ElementTree as ET
from sklearn import feature_extraction
import sklearn as skl
from nltk.stem import RSLPStemmer
import nltk
from zipfile import ZipFile
import pandas as pd
import numpy as np
import unicodedata
import os
import re
import gensim
import scipy.sparse
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import Dropout
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForPreTraining
from transformers import AutoModel

c:\Users\vitor\OneDrive\Documentos\Faculdade\inteligencia artificial\af_inteligencia_artificial\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Caminho dos arquivos extraidos do kaggle
path_dataset = r'files\classificao-de-notcias.zip'
path_db = r'db'

# Caminho dos arquivos que serão utilizados para a atividade
path_train = r'db\arquivos_competicao\arquivos_competicao\train.csv'
path_test = r'db\arquivos_competicao\arquivos_competicao\test.csv'
path_news = r'db\arquivos_competicao\arquivos_competicao\news'


In [3]:
def unzip(path, pathFolder):

    # descompacta a base de dados de notícias
    z = ZipFile(path, 'r')

    if os.path.isdir(pathFolder):
        z.extractall(pathFolder)
        z.close()
    else:
        os.mkdir(pathFolder)
        z.extractall(pathFolder)
        z.close()

    print("Arquivo descompactado com sucesso!")
    
# Antes de descompactar os arquivos valida se ja foram descompactados antes
if not os.path.isdir(path_news):
    unzip(path_dataset, path_db)
else:
    print("Arquivo já descompactado")

Arquivo já descompactado


---
## Carregando os arquivos de teste e treino

In [4]:
#Carregando os arquivos de treino
df_train = pd.read_csv(path_train)
print("Colunas do arquivo de treino: ", df_train.columns)
print("Quantidade de linhas: ", df_train.shape[0])

Colunas do arquivo de treino:  Index(['ID', 'Class'], dtype='object')
Quantidade de linhas:  2781


In [5]:
# Carregando os arquivos de teste
df_teste = pd.read_csv(path_test)
print("Colunas do arquivo de treino: ", df_teste.columns)
print("Quantidade de linhas: ", df_teste.shape[0])

Colunas do arquivo de treino:  Index(['ID'], dtype='object')
Quantidade de linhas:  1193


---
## Pré-Processamento dos Dados

In [6]:
def extract_xml_text(path_xml):
    
    """
    A função `extract_xml_text` é designada para extrair os textos dos arquivos XML
    especificados pelo `path_xml`.
    Utilizando a biblioteca ElementTree para leitura do arquivo XML 
    """
    
    # Instancia um objeto como uma árvore de análise
    tree = ET.parse(path_xml)
    
    # Obtem o elemento raiz da árvore de ánalise
    root = tree.getroot()
    
    # Encontra o elemento headline (titulo) dentro da árvore de analise
    headline = root.find('headline').text if root.find('headline') is not None else ''
    
    # Entroa todos os elementos <p> que na estrutura dos xml's contem o texto
    paragraphs = root.findall('.//p')
    
    # Junta em uma unica string, separando por espaços
    text = ' '.join([p.text for p in paragraphs if p.text is not None])

    return headline, text

def apply_extraction(df_applyed):

    """
    Essa função é responsável em aplicar as novas colunas 
    no df_applyed passado como parametro.
    """
    
    # Loop pelas linhas do df
    for idx in df_applyed.index:
        
        # atribui o valor da coluna id na variavel file
        file = df_applyed.at[idx, 'ID']
        
        # Concatena o nome do arquivo com o caminho dele
        path_xml = f"{path_news}\{file}"
        
        # Extrai o texto e titulo desse arquivo
        titulo, texto = extract_xml_text(path_xml)
        
        #Atribui esses o texto e titulos em novas colunas
        df_applyed.at[idx, 'TITULO'] = titulo
        df_applyed.at[idx, 'TEXTO'] = texto
    
    return df_applyed

def print_porcentagem(target):
    # Calcula a contagem de cada classe
    class_counts = target['Class'].value_counts()
    
    # Calcula a porcentagem de cada classe
    class_percentages = class_counts / len(target) * 100
    
    # Imprime a porcentagem de cada classe
    for cl, pct in class_percentages.items():
        print(f"Porcentagem da classe {cl}: {round(pct, 2)}%")
        
def preprocessing_portuguese(text, stemming = False, stopwords = False):
    """
    Funcao usada para tratar textos escritos na lingua portuguesa

    Parametros:
        text: variavel do tipo string que contem o texto que devera ser tratado

        stemming: variavel do tipo boolean que indica se a estemizacao deve ser aplicada ou nao

        stopwords: variavel do tipo boolean que indica se as stopwords devem ser removidas ou nao
    """

    # Lower case
    text = text.lower()

    # remove os acentos das palavras
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text)

    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)

    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')
    text = re.sub(regex, " ", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+.[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+,[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "NUMERO", text)


    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # separa o texto em palavras
    words = text.split()

    # trunca o texto para apenas 200 termos
    words = words[0:200]

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('portuguese')]
        text = " ".join( words )

    # aplica estemização
    if stemming:
        stemmer_method = RSLPStemmer()
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )

    # remove palavras compostas por apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )

    return text

In [7]:
#Aplicando a função de exracao dos textos do xml no DataFrame de treino
df_train = apply_extraction(df_train)

#Re-organizando as colunas do df
df_train = df_train[['ID', 'TITULO', 'TEXTO', 'Class']]

#Printando as classes e as suas respectativas porcentagens
print_porcentagem(df_train)

Porcentagem da classe Mercados: 74.33%
Porcentagem da classe Economia: 21.18%
Porcentagem da classe GovSocial: 3.24%
Porcentagem da classe CorpIndustrial: 1.26%


### Tratando os textos da base de dados
- Aplicada a função de estemização para a linguagem dos textos (português)
- Removendo os ascentos das palavras
- Criando um limite de 200 temrmos por palavras, para evitar que a predição do classificador seja influenciada pelo tamanho da noticia.

In [8]:
# Download the stopwords corpus
nltk.download('stopwords')

# Download the RSLPStemmer
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [9]:
# Aplicar a função ao DataFrame de treino
df_train['TEXTO'] = df_train['TEXTO'].apply(preprocessing_portuguese)

### Transformando a coluna target para valores numericos
Já que o problema a ser lidado, possui diversas classes, e o modelo a ser testado é o modelo Random Forest que não são sensiveis à ordem da categoria, para a transoformação será utilizado o Label Encoding da biblioteca sklearn, que converte cada categoria em um numero inteiro.

In [10]:
# Transformação do target
le = LabelEncoder()
df_train['Class'] = le.fit_transform(df_train['Class'])

### Separação dos dados entre dados de teste e treino

Foi utilizada a função train_test_split do scikit-learn para dividir o conjunto de dados para ter uma ideia rápida do desempenho do modelo
Os paramentros utilizados nessa separação foram:
- **features**: o conjunto de textos extraido do arquivo XML
- **target**: as classes já convertidas para numerico
- **test_size**=0.2: divisão de 20% dos dados para teste
- **random_state**=42: obter a mesma divisão toda vez que executar o código
- **stritify**=target: parametro utilizado para garantir que as proporções das classes sejam mantidas nos conjuntos de treino e teste já que as clasesses do nosso problema estão desbalanceadas 

In [11]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_train['TEXTO'],
                                                    df_train['Class'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_train['Class'])

---
## Técnicas de representação vetorial

### TF (Term-Frequency)
Utilizando a função **CountVectorizer** da biblioteca **scikit-learn** para criar uma representação vetorial dos textos utilizando a frequencia de cada palavra

Os parametros passados para essa estancia foram:
- **analyzer=word**: Define o nível de análise de tokens para palavras (word0)
- **dtype=np.int32**: Tipo de dados da matriz de contagem

In [12]:
# inicializa o modelo usado para gerar a representação TF (term frequency)
vectorizer = CountVectorizer(analyzer="word",dtype=np.int32)

# treina o modelo TF com os dados de treinamento e converte os dados de treinamento para uma array 
# que contém a frequência dos termos em cada documento (TF - term frequency)
X_train_tf = vectorizer.fit_transform(X_train)

# converte os dados de teste
X_test_tf = vectorizer.transform(X_test)


print('20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:\n')
print(vectorizer.get_feature_names_out()[0:20])

print('\nDimensão dos dados vetorizados de treino: ', X_train_tf.shape)
print('\nDimensão dos dados vetorizados de teste: ', X_test_tf.shape)

20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:

['aa' 'aaa' 'aanumero' 'aas' 'abaixo' 'abaixos' 'abaixou' 'abalaram'
 'abanadas' 'abanar' 'abandona' 'abandonado' 'abandonaram' 'abastecimento'
 'abatidas' 'abatimentos' 'abel' 'aberta' 'abertamente' 'abertas']

Dimensão dos dados vetorizados de treino:  (2224, 9509)

Dimensão dos dados vetorizados de teste:  (557, 9509)


### TF-IDF (Term Frequency-Inverse Document Frequency).
Utilizando a função **TfidfTransformer** para para converter a representação vetorial TF para TF-IDF
Os parametros utilizados nessa função foram:
- **norm=l2**: normalizando cada vetor TF-IDF para que a soma dos quadrados dos elementos seja igual a 1
- **smooth_idf**: adiciona 1 ao denominador da formula de IDF para eviter divisoes por zero
- **sublinear_tf**: aplica a sublinearização do TF

In [13]:
#Inicializa o modelo usado para gerar a representação TF-IDF
tfidf_model = skl.feature_extraction.text.TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

# Treina o modelo com os vetores de treinamento
X_train_tfidf = tfidf_model.fit_transform(X_train_tf)

# treina o modelo com os dados de teste
X_test_tfidf = tfidf_model.transform(X_test_tf)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(2224, 9509)
(557, 9509)


### Binário
utilizando a técnica de vetorização binária onde cada termo é representado como um valor binário. Ou seja, se o termo aparece no documento o valor é 1, caso contrário o valor é 0

In [14]:
X_train_bin = X_train_tf.copy()
X_test_bin = X_test_tf.copy()

#convert os dados de treino para representação binária
X_train_bin[X_train_bin!=0]=1

#convert os dados de teste para representação binária
X_test_bin[X_test_bin!=0]=1

print(X_train_bin.shape)
print(X_test_bin.shape)

(2224, 9509)
(557, 9509)


### Word Embeddings (treinada com a própria base)
Os word embeddings são uma técnica avançada de representação vetorial de palavras que capturam o significado semântico e as relações entre palavras em um espaço vetorial de alta dimensão.

In [15]:
# Tokenização dos textos de treinamento
train_documents = [text.split() for text in X_train]

# Tokenização dos textos de teste
test_documents = [text.split() for text in X_test]

print("20 primeiras palavras da primeira amostra de treino")
print(train_documents[0][0:30])

print("\n20 primeiras palavras da primeira amostra de teste")
print(test_documents[0][0:30])

20 primeiras palavras da primeira amostra de treino
['lisboa', 'NUMERO', 'mar', 'reuter', 'banco', 'de', 'portugal', 'deve', 'manter', 'inalteradas', 'todas', 'as', 'suas', 'taxas', 'directoras', 'no', 'inicio', 'do', 'proximo', 'periodo', 'de', 'reservas', 'consideram', 'sete', 'de', 'um', 'conjunto', 'de', 'NUMERO', 'economistas']

20 primeiras palavras da primeira amostra de teste
['escudo', 'fechou', 'em', 'alta', 'contra', 'marco', 'numa', 'sessao', 'volatil', 'marcada', 'pela', 'forte', 'probabilidade', 'da', 'reentrada', 'da', 'lira', 'no', 'mtc', 'do', 'sme', 'dealers', 'disseram', 'que', 'escudo', 'devera', 'manter', 'se', 'colado', 'aos']


A função **Word2Vec** foi utilizada para treinar o modelo de word embeddings utilizando os seguintes paramentros:
- **sentences**: a lista das listas de palavras tokenizadas anteriormente
- **vector_size=200**: define que cada vetor tera 200 dimensões
- **window=3**: considera 3 palavras à esquerda e 3 palavras à direita da palavra-alvo
- **min_count=1**: inclui todas as palavras que aparecem pelo menos uma vez
- **workers=4**: usa 4 threads para o treinamento

In [16]:
w2v_model = Word2Vec(train_documents,
                     vector_size=200,
                     window=3,
                     min_count=1,
                     workers=4)

print("Tamanho do vocabulario do modelo: ", len(w2v_model.wv))

Tamanho do vocabulario do modelo:  9510


In [17]:
def getDocvector(model, doc):
    # Inicializa uma lista para armazenar os vetores das palavras
    wordList = []
    
    # Itera sobre cada palavra no documento
    for word in doc:
        try:
            # Tenta obter o vetor da palavra do modelo Word2Vec
            vec = model.wv[word]
            # Adiciona o vetor da palavra à lista
            wordList.append(vec)
        except:
            # Se a palavra não estiver no vocabulário do modelo, ignora
            pass
    
    # Se a lista de vetores de palavras não estiver vazia
    if len(wordList) > 0:
        # Calcula a média dos vetores das palavras para representar o documento
        vetorMedio = np.mean(wordList, axis=0)
    else:
        # Se nenhuma palavra do documento estiver no vocabulário do modelo,
        # retorna um vetor de zeros com o mesmo tamanho dos vetores do modelo
        vetorMedio = np.zeros(model.wv.vector_size)
    
    # Retorna o vetor médio do documento
    return vetorMedio

In [18]:
def dataset2featureMatrix(dataset, embeddingModel):
    # Inicializa uma lista para armazenar os embeddings dos documentos
    X_embedding = []
    
    # Itera sobre cada documento no dataset
    for doc in dataset:
        # Obtém o vetor médio do documento usando a função getDocvector
        vec = getDocvector(embeddingModel, doc)
        # Adiciona o vetor do documento à lista de embeddings
        X_embedding.append(vec)
    
    # Converte a lista de embeddings em um array NumPy
    X_embedding = np.array(X_embedding)
    
    # Retorna a matriz de features
    return X_embedding

In [19]:
# Representação Vetorial com Word2Vec treinado na própria base
X_train_embedding = dataset2featureMatrix(train_documents, w2v_model)
X_test_embedding = dataset2featureMatrix(test_documents, w2v_model)

print(X_train_embedding.shape)
print(X_test_embedding.shape)

(2224, 200)
(557, 200)


### Word Embeddings (pré-treinadas)
Para esse caso será utilizado um modelo pré-treinado para português postado no Hugging Face, chamado [BERTimbau](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

In [20]:
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [21]:
def mean_pooling(model_output, attention_masks):
    # Extrai os embeddings dos tokens da saída do modelo
    tokenEmbeddings = model_output[0]

    # Expande a máscara de atenção para ter as mesmas dimensões que os embeddings dos tokens
    inputExpandido = attention_masks.unsqueeze(-1).expand(tokenEmbeddings.size())

    # Converte a máscara de atenção expandida para float
    inputExpandido = inputExpandido.float()

    # Calcula a média ponderada dos embeddings dos tokens
    # Multiplica os embeddings dos tokens pela máscara de atenção expandida
    # Depois soma os embeddings ao longo da dimensão dos tokens (dimensão 1)
    # Divide pela soma das máscaras de atenção para normalizar
    saida = (torch.sum(tokenEmbeddings * inputExpandido, 1) /
             torch.clamp(inputExpandido.sum(1), min=0.0000001))

    # Retorna os embeddings da sentença
    return saida

In [22]:
def get_docVec(model, tokenizer, doc):
    # Tokeniza o documento, adicionando padding e truncamento, e converte para tensores PyTorch
    encoded = tokenizer(doc,
                        padding=True,
                        truncation=True,
                        max_length=200,
                        return_tensors="pt")

    # Desabilita a computação de gradiente para economizar memória e acelerar a inferência
    with torch.no_grad():
        # Gera a saída do modelo BERT para o documento tokenizado
        model_output = model(**encoded)

    # Calcula o embedding da sentença usando a média ponderada dos embeddings dos tokens
    sentenceEmbedding = mean_pooling(model_output, encoded['attention_mask'])

    # Converte o embedding da sentença para um array NumPy
    sentenceEmbedding = sentenceEmbedding.squeeze().numpy()

    # Retorna o embedding da sentença
    return sentenceEmbedding

In [23]:
def dataset2featureMatrixBert(dataset, model, tokenizer):
    # Inicializa uma lista para armazenar os embeddings dos documentos
    X_embedding = []

    # Itera sobre cada documento no dataset
    for doc in dataset:
        # Obtém o vetor médio do documento usando a função get_docVec
        vec = get_docVec(model, tokenizer, doc)
        # Adiciona o vetor do documento à lista de embeddings
        X_embedding.append(vec)

    # Converte a lista de embeddings em um array NumPy
    return np.array(X_embedding)

In [24]:
# Geração das representações vetoriais para os conjuntos de treinamento e teste
X_train_embedding_bert = dataset2featureMatrixBert(X_train, model, tokenizer)
X_test_embedding_bert = dataset2featureMatrixBert(X_test, model, tokenizer)

print("Dimensões dos embeddings de treinamento:", X_train_embedding_bert.shape)
print("Dimensões dos embeddings de teste:", X_test_embedding_bert.shape)

Dimensões dos embeddings de treinamento: (2224, 768)
Dimensões dos embeddings de teste: (557, 768)


---
## Treinamento do modelo Random Forest

In [39]:
def classificar(X_train, X_test, Y_train, Y_test):
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)
    rfc.fit(X_train, Y_train)
    
    # Fazendo previsões no conjunto de teste
    y_pred = rfc.predict(X_test)

    # Avaliando a acurácia
    accuracy = accuracy_score(Y_test, y_pred)
    print(f"Acurácia: {accuracy:.4f}")


In [40]:
print("\n\nTreinando com o formato TF")
model_tf = classificar(X_train_tf, X_test_tf, y_train, y_test)

print("\n\nTreinando com o formato binário")
model_bin = classificar(X_train_bin, X_test_bin, y_train, y_test)

print("\n\nTreinando com o formato TF-IDF")
model_tfidf = classificar(X_train_tfidf, X_test_tfidf, y_train, y_test)

print("\n\nTreinando com word embeddings")
model_embedding = classificar(X_train_embedding, X_test_embedding, y_train, y_test)

print("\n\nTreinando com word embeddings pré-treinados")
model_embedding_bert = classificar(X_train_embedding_bert, X_test_embedding_bert, y_train, y_test)



Treinando com o formato TF
Acurácia: 0.9425


Treinando com o formato binário
Acurácia: 0.9443


Treinando com o formato TF-IDF
Acurácia: 0.9408


Treinando com word embeddings
Acurácia: 0.9408


Treinando com word embeddings pré-treinados
Acurácia: 0.9282


---
## Escolha dos Hiperparâmetros

In [42]:
# from sklearn.model_selection import GridSearchCV

# # Definindo os parâmetros para a busca em grade
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# # Inicializando o RandomForestClassifier
# clf = RandomForestClassifier(random_state=42)

# # Inicializando o GridSearchCV
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# # Ajustando o modelo nos dados de treinamento
# grid_search.fit(X_train_embedding_bert, y_train)

# # Obtendo os melhores parâmetros
# best_params = grid_search.best_params_
# print("Melhores parâmetros:", best_params)

# # Treinando o modelo com os melhores parâmetros
# best_clf = RandomForestClassifier(**best_params, random_state=42)
# best_clf.fit(X_train_embedding_bert, y_train)

# # Avaliando o modelo
# y_pred = best_clf.predict(X_test_embedding_bert)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Acurácia com os melhores parâmetros: {accuracy:.4f}")
